In [ ]:
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTShadowClient
import pandas as pd
import time
SHADOW_CLIENT = "A0195028B_3"
HOST_NAME = "a2h9i547jb98l3-ats.iot.us-west-2.amazonaws.com"
ROOT_CA = "/Users/yuxingli/Desktop/IoT/Amazon_Root_CA_1.pem"
PRIVATE_KEY = "/Users/yuxingli/Desktop/IoT/69756c467d-private.pem.key"
CERT_FILE = "/Users/yuxingli/Desktop/IoT/69756c467d-certificate.pem.crt"
SHADOW_HANDLER = "A0195028B_3"
# Create, configure, and connect a shadow client.
myShadowClient = AWSIoTMQTTShadowClient(SHADOW_CLIENT)
myShadowClient.configureEndpoint(HOST_NAME, 8883)
myShadowClient.configureCredentials(ROOT_CA, PRIVATE_KEY, CERT_FILE)
myShadowClient.configureConnectDisconnectTimeout(10)
myShadowClient.configureMQTTOperationTimeout(5)
myShadowClient.connect()

# Create a programmatic representation of the shadow.
myDeviceShadow = myShadowClient.createShadowHandlerWithName(SHADOW_HANDLER, True)

def myShadowUpdateCallback(payload, responseStatus, token):
    print('UPDATE: $aws/things/' + SHADOW_HANDLER +'/shadow/update/#')
    print("responseStatus = " + responseStatus)

CPIS = pd.read_csv('/Users/yuxingli/Desktop/IoT/consumer-price-index-base-year-2014-100-monthly.csv')
CPI = pd.DataFrame()
CPI = CPI.append([(CPIS.iloc[i*4]) for i in range(703)])
CPI.index = pd.Series(range(703))
CPI = CPI.drop(['level_1'], axis=1)
for i in range(703):
    j = CPI[i:i+1]
    j = j.to_json(orient="records")
    myDeviceShadow.shadowUpdate(j[1:-1], myShadowUpdateCallback, 5)
    time.sleep(0.2)

In [ ]:
from __future__ import print_function 
import json
import decimal


class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, decimal.Decimal):
            return str(o)
        return super(DecimalEncoder, self).default(o)


dynamodb = boto3.resource('dynamodb', region_name='us-west-2')
table = dynamodb.Table('CPI')
CPI_accrpted = {'month':[],'value':[]}
response = table.scan()
for i in response['Items']:
    CPI_accrpted['month'].append(i['month'])
    CPI_accrpted['value'].append(float(i['value']))
CPI_pd = pd.DataFrame(CPI_accrpted)
CPI_pd.sort_values("month",inplace=True)
CPI_pd.index = pd.Series(range(703))
CPI_pd.transpose().head()

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
CPI_se = pd.Series(CPI_pd["value"])
CPI_se.index = CPI_pd["month"]
plt.figure(1,dpi=240)
pl = CPI_se.plot(figsize=(8,6),color = 'deepskyblue',kind='line')
plt.xlabel("Month")
plt.ylabel("Value")
plt.title("Consumer Price Index, Monthly, Base Year 2014=100")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(2,dpi=240)
pl = CPI_se.plot(figsize=(8,6),color = 'deepskyblue',kind='hist')
CPI_ave = CPI_se.count()/10
plt.plot([22,102],[CPI_ave,CPI_ave],color="darkorange",linestyle="--")
plt.xlabel("Value")
plt.ylabel("Sum")
plt.title("Consumer Price Index, Monthly, Base Year 2014=100")
plt.grid(True)
plt.show()

In [ ]:
plt.figure(3,dpi = 240)
CPI_gradient = CPI_se.diff()/CPI_a.shift(1)
CPI_gradient_mean = CPI_gradient.mean()
CPI_gradient.plot(color = "deepskyblue")
plt.plot([0,703],[0,0],color = "black")
plt.plot([0,703],[CPI_gradient_mean,CPI_gradient_mean],color="darkorange",linestyle="--")
plt.text(280, 0.01, "Average rate of\nchange: "+str(round(CPI_gradient_mean*100, 2))+'%', size = 10)
plt.xlabel("Month")
plt.ylabel("Change rate")
plt.show()

In [ ]:
CPI_v = CPI_gradient.values - CPI_gradient_mean
CPI_v = CPI_v[1:]
CPI_v = np.append(CPI_v,[0 for i in range(702)])
transformed = np.fft.fftshift(np.abs(np.fft.fft(CPI_v)))
transformed = transformed[702:]
plt.figure(4,dpi = 240)
plt.plot([i/701 for i in range(702)],transformed,color = "deepskyblue")
plt.grid(True)
plt.xlabel("Frequency (1/month)")
plt.ylabel("Amplitude")
plt.show()